In [1]:
from cerebro.backend import SparkBackend
from cerebro.keras import SparkEstimator

# datas storage for intermediate data and model artifacts.
from cerebro.storage import LocalStore, HDFSStore

# Model selection/AutoML methods.
from cerebro.tune import GridSearch, RandomSearch, TPESearch

# Utility functions for specifying the search space.
from cerebro.tune import hp_choice, hp_uniform, hp_quniform, hp_loguniform, hp_qloguniform

import tensorflow as tf
# tf.config.run_functions_eagerly(True)

from pyspark.sql import SparkSession
import numpy as np
import autokeras as ak

spark = SparkSession \
    .builder \
    .appName("Cerebro Iris") \
    .getOrCreate()

...

backend = SparkBackend(spark_context=spark.sparkContext, num_workers=1)
store = LocalStore(prefix_path='/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/experiments')

from pyspark.ml.feature import OneHotEncoderEstimator

df = spark.read.csv("/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/Iris_clean.csv", header=True, inferSchema=True)

encoder = OneHotEncoderEstimator(dropLast=False)
encoder.setInputCols(["Species"])
encoder.setOutputCols(["Species_OHE"])

encoder_model = encoder.fit(df)
encoded = encoder_model.transform(df)

feature_columns=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
label_columns=['Species_OHE']

# Initialize input DataFrames.
# You can download sample dataset from https://apache.googlesource.com/spark/+/master/data/mllib/sample_libsvm_data.txt

train_df, test_df = encoded.randomSplit([0.8, 0.2], 100)

21/11/22 06:01:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/22 06:01:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/11/22 06:01:12 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
21/11/22 06:01:12 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


CEREBRO => Time: 2021-11-22 06:01:13, Running 1 Workers


In [2]:
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.model_selection import train_test_split

# pddf = df.toPandas()
# train_df, test_df = train_test_split(pddf, test_size=0.2, random_state=42, shuffle=True)
# train_df, test_df = df.randomSplit([0.8, 0.2])
# train = train_df.toPandas()
# test = test_df.toPandas()

# x_train = train_df[feature_columns]
# y_train = train_df[['Species']]

# enc = OneHotEncoder()
# x = np.array(x_train)
# x = [x[:,i,np.newaxis] for i in range(x.shape[1])]
# y = enc.fit_transform(y_train).toarray()

# x_test = test_df[feature_columns]
# y_test = test_df[['Species']]

# y_test = enc.fit_transform(y_test).toarray()
# x_test = np.array(x_test)
# x_test = [x_test[:,i,np.newaxis] for i in range(x_test.shape[1])]

In [3]:
x = np.array(train_df.select(feature_columns).collect())
y = np.array(train_df.select(label_columns).collect())
x = [x[:,i,np.newaxis] for i in range(x.shape[1])]
y = np.squeeze(y,1)
print(y.shape)

x_test = np.array(test_df.select(feature_columns).collect())
y_test = np.array(test_df.select(label_columns).collect())
x_test = [x_test[:,i,np.newaxis] for i in range(x_test.shape[1])]
y_test = np.squeeze(y_test,1)
print(y_test.shape)

(120, 3)
(30, 3)


In [4]:
input_node = [ak.StructuredDataInput() for c in feature_columns]
# embeddings = [ak.StructuredDataBlock()(innode) for innode in input_node]
# output_node = ak.Merge()([embeddings])
concat = ak.Merge()(input_node)
output_node = ak.DenseBlock()(concat)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    inputs=input_node, 
    outputs=output_node, 
    max_trials=20, 
    tuner='random',
    objective="val_loss",
    seed=100,
    overwrite=True
)

In [5]:
# dataset, validation_data = clf._convert_to_dataset(
#             x=x, y=y, validation_data=None, batch_size=32
#         )
# clf._analyze_data(dataset)
# clf._build_hyper_pipeline(dataset)

In [6]:
# hp = clf.tuner.oracle.get_space()
# clf.tuner._prepare_model_build(hp, x=dataset)
# clf.tuner.hypermodel.build(hp)
# clf.tuner.oracle.update_space(hp)

In [7]:
# trial = clf.tuner.oracle.create_trial(clf.tuner.tuner_id)
# trial.hyperparameters.values

In [8]:
# clf.tuner.oracle._tried_so_far

In [9]:
# clf.tuner.oracle.get_space().space

In [10]:
clf.fit(
    x,
    y,
    # The name of the label column.
    epochs=10,
)

Trial 20 Complete [00h 00m 02s]
val_loss: 0.5947383046150208

Best val_loss So Far: 0.36133790016174316
Total elapsed time: 00h 00m 59s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
4/4 [==============================] - 0s 5ms/step - loss: 1.9923 - accuracy: 0.3250
Epoch 2/10
4/4 [==============================] - 0s 5ms/step - loss: 1.2646 - accuracy: 0.3123
Epoch 3/10
4/4 [==============================] - 0s 4ms/step - loss: 0.9194 - accuracy: 0.5104
Epoch 4/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6687 - accuracy: 0.7100
Epoch 5/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6019 - accuracy: 0.8075
Epoch 6/10
4/4 [==============================] - 0s 4ms/step - loss: 0.6156 - accuracy: 0.7454
Epoch 7/10
4/4 [==============================] - 0s 4ms/step - loss: 0.4541 - accuracy: 0.8617
Epoch 8/10
4/4 [==============================] - 0s 4ms/step - loss: 0.4707 - accuracy: 0.7746
Epoch 9/10
4/4 [==============================] - 0s 4ms/s

2021-11-22 06:02:18.877548: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./auto_model/best_model/assets


In [11]:
model = clf.tuner.get_best_model()

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
______________________________________________________________________________________________

In [13]:
clf.evaluate(x_test, y_test)

1/1 [==============================] - 0s 186ms/step - loss: 0.2815 - accuracy: 0.8667


[0.28146201372146606, 0.8666666746139526]

In [14]:
model.optimizer.get_config()

{'name': 'SGD',
 'learning_rate': 0.10000000149011612,
 'decay': 0.0,
 'momentum': 0.0,
 'nesterov': False}

In [15]:
pd_train = train_df.toPandas()
pd_test = test_df.toPandas()

In [16]:
pd_train["Species"].value_counts()

2    43
1    42
0    35
Name: Species, dtype: int64

In [17]:
pd_test["Species"].value_counts()

0    15
1     8
2     7
Name: Species, dtype: int64

In [19]:
clf.tuner.oracle._tried_so_far

{'1d13b9daf356c746e74fb36a22be5cba',
 '1df78473d7eb0beadcebc4a6665853b8',
 '217b94f91ceb2d7196f3ce3e073be6ba',
 '46f5872d280a709859673af8d389b8ca',
 '4ea89569c6ecd802ce30aa61d9b74a92',
 '5b29e17452483172b48198030a343698',
 '67925b517d7d493145dfa536310c29d0',
 '70b015c7e186ce401125f73bccb722b0',
 '72a3195d68017860075009b1dacdcbf3',
 '7e69c12d0c876e0dcbf1df948001878e',
 '87f4a873b531a01281ff91149567ff7b',
 'b741a21bd822202fca8a9f39fb55ddbf',
 'ce4abdf3ec7cdf8b7830444fc366a589',
 'dc04fd1487b18abd83df22678963c725',
 'dfec8137a53ce41991f8931cef4e6d2d',
 'e600389267e922d0fdf7b860c0a73861',
 'e80967daa9e2578e6dae2ed580466650',
 'f19b7e6d721e1e1d94f9dd74db31e041',
 'f23dfd80a5340a38a7996bdb78dc4475',
 'fb3b254380b6fd14eed338431fb4a1a5'}